In [1]:
import pandas as pd
from oaklib import get_adapter

In [2]:
RESULTS_DIR = "../results/manual"

In [3]:
from c3p.datamodel import Dataset

BENCH_DIR = "../results/2025/benchmark"
with open(f"{BENCH_DIR}/dataset.json", "r") as f:
    dataset = Dataset.model_validate_json(f.read())

In [4]:
name2smiles = {s.name: s.smiles for s in dataset.structures}

In [5]:
chebi = get_adapter("sqlite:obo:chebi")

In [6]:
# check the data...
BETALAMIC_ACID = "CHEBI:27483"
ALKALOID = "CHEBI:22315"


In [7]:
def get_smiles_from_chebi(chebi_id: str):
    return chebi.entity_metadata_map(chebi_id)["obo:chebi/smiles"][0]

BETALAMIC_ACID_SMILES = get_smiles_from_chebi(BETALAMIC_ACID)
BETALAMIC_ACID_SMILES

'OC(=O)[C@@H]1CC(=C/C=O)\\C=C(N1)C(O)=O'

In [8]:
alkaloid_cls = dataset.get_chemical_class_by_id(ALKALOID)
alkaloid_cls.name

'alkaloid'

In [9]:
BETALAMIC_ACID_SMILES in alkaloid_cls.all_positive_examples

True

In [10]:
curator_df = pd.read_csv(f"{RESULTS_DIR}/manual-evals-results.tsv", sep="\t")

In [11]:
curator_df = curator_df[curator_df["chemical_class_id"].notna()]
curator_df

,chemical_class_id,chemical_class_name,smiles,fixed_smiles,structure_name,model=citeseek-gpt-4o_response,model=claude-sonnet_response,model=deepseek-chat_response,model=gpt-3.5-turbo_response,model=gpt-4o_response,Curator: is_member?,Curator: comments
0,CHEBI:20706,6-aminopurines,O=NN1CCNCC1,No change,N-Mononitrosopiperazine,"NO, the statement is incorrect. N-Mononitros...",NO. N-Mononitrosopiperazine (with SMILES 'O=NN...,NO. The structure N-Mononitrosopiperazine (wit...,NO Explanation: N-Mononitrosopiperazine is no...,"NO, the provided statement is incorrect. N-M...",No,it is a nitroso compound and a piperazine
1,CHEBI:20706,6-aminopurines,O1C(CCC1)C(=O)N2CCNCC2,No change,1-(Tetrahydro-2-furoyl)piperazine,"NO, the statement is not true. 1-(Tetrahydro-...",NO The provided statement is false. The struc...,NO. The structure 1-(Tetrahydro-2-furoyl)piper...,"NO Explanation: The structure provided, 1-(Te...","NO, 1-(Tetrahydro-2-furoyl)piperazine is not a...",No,it is a piperazine and a tetrahydrofuran
2,CHEBI:20706,6-aminopurines,NC1=NC=NC2=C1N=CN2[C@@H]3O[C@H](CO)[C@@H](OP(O...,No change,"5'-hydroxy-diadenylate with a cyclic 2',3' pho...","YES, the provided statement is true. The stru...","YES, the provided statement is true. Explanat...","YES. The structure described, 5'-hydroxy-diade...",YES The provided structure is indeed a type o...,"YES, The provided structure '5'-hydroxy-diaden...",Yes,NaN
3,CHEBI:47622,acetate ester,CCCCCCCCCCCCCC(=O)OC[C@H](COP([O-])(=O)OC[C@@H...,CCCCCCCCCCCCCC(=O)OC[C@H](COP([O-])(=O)OC[C@@H...,1-myristoyl-2-oleoyl-sn-glycero-3-phosphatidyl...,"NO, the statement is not true. The structure...","NO, the provided statement is not true. Expla...",NO. The structure 1-myristoyl-2-oleoyl-sn-glyc...,NO Explanation: The provided SMILES string 'C...,"NO, the provided statement is incorrect. 1-m...",No,It is a carboxylic ester
4,CHEBI:17984,acyl-CoA,CCC=C/CC=C/CC=C/CC=C/CC=C/CCCCCCCCCC(=O)SCCNC(...,CC\C=C/C\C=C/C\C=C/C\C=C/C\C=C/CCCCCCCCCC(=O)S...,"(11Z,14Z,17Z,20Z,23Z)-hexacosapentaenoyl-CoA(4-)","YES, the provided statement is true. The SMI...","YES, the provided statement is true. The stru...","YES. The structure (11Z,14Z,17Z,20Z,23Z)-hexac...","YES, the statement is true. The provided compo...","YES, the statement is true. The structure desc...",No,It is an acyl CoA(4-). Its net charge is -4 wh...
...,...,...,...,...,...,...,...,...,...,...,...,...
217,CHEBI:61910,very long-chain fatty acyl-CoA,CCCCCCCCCCCCCCCCCCCCCCCCC(C(O)=O)C(=O)SCCNC(=O...,No change,2-carboxyhexacosanoyl-CoA,"YES, the provided statement is true. The struc...",YES The provided SMILES string 'CCCCCCCCCCCCC...,YES. The structure 2-carboxyhexacosanoyl-CoA i...,YES Explanation: The provided SMILES structur...,"YES, the provided statement is true. The compo...",Yes,NaN
218,CHEBI:10036,wax ester,CCCCCCCCCCCCCCCC(=O)OCC(CCCCCCCCCCCCCC)OC(=O)C...,No change,2-palmitoyloxypalmityl palmitate,"YES, the provided statement is true. The given...","YES, the provided statement is true. 2-palmit...",YES. The structure 2-palmitoyloxypalmityl palm...,YES Explanation: The provided structure (2-pa...,YES. The structure 2-palmitoyloxypalmityl pa...,Yes,it is also a diester
219,CHEBI:10036,wax ester,CCCCCCCCCCOC(=O)CCCCCCCC=C/CCCCCCCC,CCCCCCCCCCOC(=O)CCCCCCC\C=C/CCCCCCCC,decyl oleate,"YES, the structure known as decyl oleate is a ...","YES, the provided statement is true. Decyl ol...",YES. Decyl oleate is a wax ester. It is formed...,YES Decyl oleate is a type of wax ester. It i...,"YES, the provided statement is true. Decyl ole...",Yes,NaN
220,CHEBI:10036,wax ester,OC(CCCCCCCCOC(=O)CCCCCCCCCCCCCCCCCCCCC)CCCC,No change,9-Hydroxytridecyl docosanoate,"YES, the statement is true. The structure 9-Hy...","YES, the provided statement is correct. 9-Hydr...",YES. The structure 9-Hydroxytridecyl docosanoa...,YES Explanation: The provided structure of 9-...,"YES, the statement is true. The structure know...",Yes,NaN


In [12]:
# fix SMILES
fixed = {}

def fix_smiles(name: str, curr_smiles: str):
    if name not in name2smiles:
        raise ValueError(f"Name not in dataset: {name}")
    fixed_smiles = name2smiles[name]
    if fixed_smiles != curr_smiles:
        fixed[name] = fixed_smiles
        # check if they differ other than backslashes
        if fixed_smiles.replace("\\", "") != curr_smiles.replace("\\", ""):
            raise ValueError(f"SMILES differ: {fixed_smiles} != {curr_smiles}")
    return fixed_smiles

In [13]:
curator_df["smiles"] = curator_df.apply(lambda row: fix_smiles(row["structure_name"], row["smiles"]), axis=1)

In [14]:
len(fixed)

50

In [15]:
# turned dict into a df
fixed_df = pd.DataFrame(list(fixed.items()), columns=["name", "smiles"])
fixed_df.to_csv(f"{RESULTS_DIR}/fixed-smiles.csv", index=False)
fixed_df.to_excel(f"{RESULTS_DIR}/fixed-smiles.xlsx", index=False)

In [88]:
CUR_IS_MEMBER = "Curator: is_member?"
CUR_COMMENTS = "Curator: comments"
CITESEEK = "model=citeseek-gpt-4o_response"

In [93]:
curator_df[CITESEEK]

0      NO, the statement is incorrect.   N-Mononitros...
1      NO, the statement is not true.  1-(Tetrahydro-...
2      YES, the provided statement is true.  The stru...
3      NO, the statement is not true.   The structure...
4      YES, the provided statement is true.   The SMI...
                             ...                        
217    YES, the provided statement is true. The struc...
218    YES, the provided statement is true. The given...
219    YES, the structure known as decyl oleate is a ...
220    YES, the statement is true. The structure 9-Hy...
221    YES, the provided structure dodecyl dodecanoat...
Name: model=citeseek-gpt-4o_response, Length: 222, dtype: object

In [17]:
curator_df.columns

Index(['chemical_class_id', 'chemical_class_name', 'smiles', 'fixed_smiles',
       'structure_name', 'model=citeseek-gpt-4o_response',
       'model=claude-sonnet_response', 'model=deepseek-chat_response',
       'model=gpt-3.5-turbo_response', 'model=gpt-4o_response',
       'Curator: is_member?', 'Curator: comments'],
      dtype='object')

In [18]:
curator_df.describe()

,chemical_class_id,chemical_class_name,smiles,fixed_smiles,structure_name,model=citeseek-gpt-4o_response,model=claude-sonnet_response,model=deepseek-chat_response,model=gpt-3.5-turbo_response,model=gpt-4o_response,Curator: is_member?,Curator: comments
count,222,222,222,222,222,222,222,222,222,222,222,93
unique,94,94,201,51,201,222,222,222,222,222,3,81
top,CHEBI:61051,lipid hydroperoxide,CCCCCCCCCCCCCC(=O)OC[C@H](COP([O-])(=O)OC[C@@H...,No change,1-myristoyl-2-oleoyl-sn-glycero-3-phosphatidyl...,"NO, the statement is incorrect. N-Mononitros...",NO. N-Mononitrosopiperazine (with SMILES 'O=NN...,NO. The structure N-Mononitrosopiperazine (wit...,NO Explanation: N-Mononitrosopiperazine is no...,"NO, the provided statement is incorrect. N-M...",No,I may have said yes if i had ignored the charg...
freq,6,6,5,163,5,1,1,1,1,1,150,3


In [19]:
curator_df[curator_df[CUR_COMMENTS].notna()][["chemical_class_name", "structure_name", CUR_IS_MEMBER, CUR_COMMENTS]]

,chemical_class_name,structure_name,Curator: is_member?,Curator: comments
0,6-aminopurines,N-Mononitrosopiperazine,No,it is a nitroso compound and a piperazine
1,6-aminopurines,1-(Tetrahydro-2-furoyl)piperazine,No,it is a piperazine and a tetrahydrofuran
3,acetate ester,1-myristoyl-2-oleoyl-sn-glycero-3-phosphatidyl...,No,It is a carboxylic ester
4,acyl-CoA,"(11Z,14Z,17Z,20Z,23Z)-hexacosapentaenoyl-CoA(4-)",No,It is an acyl CoA(4-). Its net charge is -4 wh...
5,acyl-CoA,N-Mononitrosopiperazine,No,It is a piperazine
...,...,...,...,...
200,secondary alcohol,"1,2-bis(4-hydroxyphenyl)propan-2-ol",No,It seems like a tertiary alcohol (more precise...
202,steroid,"(5E,7E,10E)-(3S)-19-methyl-9,10-seco-5,7,10(19...",Yes,It is a secosteroid (a subclass of steroids)
203,steroid,"(20S,24R)-dihydroxyvitamin D3",Yes,It is a secosteroid (a subclass of steroids)
213,triterpenoid saponin,Diacetylpyxinol,No,structure is missing the glycoside group


In [20]:
def col_summary(col):
    return curator_df.groupby(col).size()

In [21]:
curator_df[CUR_IS_MEMBER].unique()

array(['No', 'Yes', 'Unknown'], dtype=object)

In [22]:
curator_df[curator_df[CUR_IS_MEMBER] == "Unknown"]

,chemical_class_id,chemical_class_name,smiles,fixed_smiles,structure_name,model=citeseek-gpt-4o_response,model=claude-sonnet_response,model=deepseek-chat_response,model=gpt-3.5-turbo_response,model=gpt-4o_response,Curator: is_member?,Curator: comments
83,CHEBI:72544,flavonoids,C12=C(C3=C(C(=CC(O3)=O)C4=CC=CC=C4)C5=C1C=CC(O...,No change,inophyllolide chromanol,"NO, the structure known as inophyllolide chrom...",NO. The provided structure of inophyllolide ch...,NO. The structure of inophyllolide chromanol (...,NO Explanation: Inophyllolide chromanol is no...,"NO, the statement is not true. Inophyllolide...",Unknown,could potentially be a flavonoid derivative
124,CHEBI:35381,monosaccharide,[H][C@@]1(OC(=O)C(O)=C1O)[C@@H](O)CO,No change,L-ascorbic acid,"NO, the provided statement is not true. L-asc...",NO. The structure of L-ascorbic acid (vitamin ...,NO. L-ascorbic acid (vitamin C) is not a monos...,NO Explanation: L-ascorbic acid is not consid...,"NO, L-ascorbic acid is not a monosaccharide. ...",Unknown,There is some confusion in the literature rega...
126,CHEBI:25409,monoterpenoid,OC(=O)C[C@@H](CC/C=C(\C)/C(O)=O)C,OC(=O)C[C@@H](CC/C=C(\C)/C(O)=O)C,"2,6R-Dimethyl-2E-octen-1,8-dioic acid","NO, the statement is incorrect. The structur...","NO, the provided statement is not true. Expla...","NO. The structure 2,6R-Dimethyl-2E-octen-1,8-d...",NO Explanation: The provided structure is not...,"NO, the provided statement is not true. The st...",Unknown,I am unable to decide by looking at its chemic...


In [23]:
curator_df.aggregate({CUR_IS_MEMBER: ["count"]})

,Curator: is_member?
count,222


In [24]:
def classifies_in_benchmark(row: dict):
    cc = dataset.get_chemical_class_by_id(row["chemical_class_id"])
    return row["smiles"] in cc.all_positive_examples

curator_df["in_benchmark"] = curator_df.apply(classifies_in_benchmark, axis=1)
    

In [25]:
curator_df[curator_df["smiles"] == BETALAMIC_ACID_SMILES]

,chemical_class_id,chemical_class_name,smiles,fixed_smiles,structure_name,model=citeseek-gpt-4o_response,model=claude-sonnet_response,model=deepseek-chat_response,model=gpt-3.5-turbo_response,model=gpt-4o_response,Curator: is_member?,Curator: comments,in_benchmark
15,CHEBI:22315,alkaloid,OC(=O)[C@@H]1CC(=C/C=O)\C=C(N1)C(O)=O,OC(=O)[C@@H]1CC(=C/C=O)\C=C(N1)C(O)=O,betalamic acid,"NO, the provided statement is not true. Beta...",NO. Betalamic acid (SMILES: OC(=O)[C@@H]1CC(=C...,NO. Betalamic acid is not an alkaloid. While i...,NO Betalamic acid is not typically considered...,"NO, betalamic acid is not classified as an alk...",No,is a dicarboxylic acid and an aldehyde.,True


In [26]:
betalamic_acid_smiles_retrieved = curator_df[curator_df["structure_name"] == "betalamic acid"]["smiles"].unique()[0]
betalamic_acid_smiles_retrieved

'OC(=O)[C@@H]1CC(=C/C=O)\\C=C(N1)C(O)=O'

In [27]:
betalamic_acid_smiles_retrieved, BETALAMIC_ACID_SMILES, betalamic_acid_smiles_retrieved == BETALAMIC_ACID_SMILES

('OC(=O)[C@@H]1CC(=C/C=O)\\C=C(N1)C(O)=O',
 'OC(=O)[C@@H]1CC(=C/C=O)\\C=C(N1)C(O)=O',
 True)

In [28]:
assert betalamic_acid_smiles_retrieved == BETALAMIC_ACID_SMILES

In [29]:
print(betalamic_acid_smiles_retrieved, BETALAMIC_ACID_SMILES)

OC(=O)[C@@H]1CC(=C/C=O)\C=C(N1)C(O)=O OC(=O)[C@@H]1CC(=C/C=O)\C=C(N1)C(O)=O


In [30]:
col_summary("in_benchmark")

in_benchmark
False    124
True      98
dtype: int64

In [31]:
from oaklib.datamodels.vocabulary import IS_A


def classifies_in_chebi(row: dict):
    inst_ids = chebi.curies_by_label(row["structure_name"])
    if len(inst_ids) != 1:
        raise ValueError(f"Ambiguous {row} ==> {inst_ids}")
    inst_id = inst_ids[0]
    return row["chemical_class_id"] in list(chebi.ancestors(inst_id, [IS_A]))

In [32]:
curator_df["in_chebi"] = curator_df.apply(classifies_in_chebi, axis=1)


In [33]:
col_summary("in_chebi")

in_chebi
False    124
True      98
dtype: int64

In [34]:
curator_df["agreement_benchmark_chebi"] = curator_df.apply(lambda row: row["in_chebi"] == row["in_benchmark"], axis=1)


In [35]:
def agrees(yn, v):
    if yn == "Unknown":
        return None
    return (yn == "Yes") == v
    
curator_df["agreement_curator_benchmark"] = curator_df.apply(lambda row: agrees(row[CUR_IS_MEMBER], row["in_benchmark"]), axis=1)
curator_df["agreement_curator_chebi"] = curator_df.apply(lambda row: agrees(row[CUR_IS_MEMBER], row["in_chebi"]), axis=1)


In [36]:
curator_df.groupby(["agreement_curator_benchmark"]).size()

agreement_curator_benchmark
False     94
True     125
dtype: int64

In [37]:
curator_df.groupby(["agreement_curator_chebi"]).size()


agreement_curator_chebi
False     94
True     125
dtype: int64

In [38]:
# get counts of each combination
curator_df.groupby([CUR_IS_MEMBER, "in_benchmark"]).size()

Curator: is_member?  in_benchmark
No                   False           90
                     True            60
Unknown              True             3
Yes                  False           34
                     True            35
dtype: int64

In [39]:
curator_df.groupby(["agreement_curator_benchmark"]).size()


agreement_curator_benchmark
False     94
True     125
dtype: int64

In [82]:
gby = curator_df.groupby(["agreement_curator_benchmark"])
gby.size() / len(curator_df)

agreement_curator_benchmark
False    0.423423
True     0.563063
dtype: float64

In [40]:
%load_ext autoreload
%autoreload 2

In [41]:
from pathlib import Path
from c3p.classifier import classify

smiles_strs = list(curator_df["smiles"].unique())
chemical_class_names = list(curator_df["chemical_class_name"].unique())


In [42]:
len(smiles_strs), len(chemical_class_names), len(smiles_strs) * len(chemical_class_names)

(201, 94, 18894)

In [63]:
results = list(classify(smiles_strs, program_directory=Path("../results/2025/ensemble-9/programs"), chemical_classes=chemical_class_names, strict=True))

In [64]:
len(results)

18894

In [65]:
results_by_pair = {(r.class_name, r.input_smiles): r.is_match for r in results}

In [66]:
#list(results_by_pair.keys())[0:10]

In [108]:
curator_df["in_c3p"] = curator_df.apply(lambda row: results_by_pair[row["chemical_class_name"], row["smiles"]], axis=1)


In [109]:
curator_df.groupby(["in_c3p"]).size()

in_c3p
False    155
True      67
dtype: int64

In [69]:
curator_df["agreement_c3p_curator"] = curator_df.apply(lambda row: agrees(row[CUR_IS_MEMBER], row["in_c3p"]), axis=1)


In [70]:
#curator_df["c3p_correct"]

In [71]:
curator_df.groupby(["agreement_c3p_curator"]).size()

agreement_c3p_curator
False     51
True     168
dtype: int64

In [72]:
curator_df.groupby(["agreement_c3p_curator"]).size() / len(curator_df)


agreement_c3p_curator
False    0.229730
True     0.756757
dtype: float64

In [73]:
curator_df_filtered = curator_df[curator_df[CUR_IS_MEMBER] != "Unknown"]

In [110]:
curator_df_filtered.groupby(["agreement_c3p_curator"]).size() / len(curator_df_filtered)

agreement_c3p_curator
False    0.232877
True     0.767123
dtype: float64

In [114]:
curator_df_filtered.apply(lambda row: row[CUR_IS_MEMBER] == "Yes", axis=1)

0      False
1      False
2       True
3      False
4      False
       ...  
217     True
218     True
219     True
220     True
221     True
Length: 219, dtype: bool

In [115]:
curator_df_filtered["cur"] = curator_df_filtered.apply(lambda row: row[CUR_IS_MEMBER] == "Yes", axis=1)

In [116]:
curator_df_filtered["cur"] 

0      False
1      False
2       True
3      False
4      False
       ...  
217     True
218     True
219     True
220     True
221     True
Name: cur, Length: 219, dtype: bool

In [95]:
curator_df_filtered[CITESEEK]

0      NO, the statement is incorrect.   N-Mononitros...
1      NO, the statement is not true.  1-(Tetrahydro-...
2      YES, the provided statement is true.  The stru...
3      NO, the statement is not true.   The structure...
4      YES, the provided statement is true.   The SMI...
                             ...                        
217    YES, the provided statement is true. The struc...
218    YES, the provided statement is true. The given...
219    YES, the structure known as decyl oleate is a ...
220    YES, the statement is true. The structure 9-Hy...
221    YES, the provided structure dodecyl dodecanoat...
Name: model=citeseek-gpt-4o_response, Length: 219, dtype: object

In [120]:
curator_df_filtered["llm"] = curator_df_filtered.apply(lambda row: row.get(CITESEEK, "").startswith("YES"), axis=1)

In [122]:
curator_df_filtered["llm_agrees_with_cur"] = curator_df_filtered.apply(lambda row: row["cur"] == row["llm"], axis=1)

In [123]:
curator_df_filtered.groupby(["llm_agrees_with_cur"]).size() / len(curator_df_filtered)

llm_agrees_with_cur
False    0.155251
True     0.844749
dtype: float64

In [121]:
curator_df_filtered.groupby(["cur", "llm"]).size()

cur    llm  
False  False    123
       True      27
True   False      7
       True      62
dtype: int64

In [75]:
curator_df.groupby([CUR_IS_MEMBER, "in_benchmark", "in_c3p", "in_chebi"]).size()


Curator: is_member?  in_benchmark  in_c3p  in_chebi
No                   False         False   False       72
                                   True    False       18
                     True          False   True        55
                                   True    True         5
Unknown              True          True    True         3
Yes                  False         False   False        5
                                   True    False       29
                     True          False   True        23
                                   True    True        12
dtype: int64

In [76]:
curator_df[(curator_df["in_benchmark"] == False) & (curator_df["in_c3p"] == False)] 

,chemical_class_id,chemical_class_name,smiles,fixed_smiles,structure_name,model=citeseek-gpt-4o_response,model=claude-sonnet_response,model=deepseek-chat_response,model=gpt-3.5-turbo_response,model=gpt-4o_response,Curator: is_member?,Curator: comments,in_benchmark,in_chebi,agreement_benchmark_chebi,agreement_curator_benchmark,agreement_curator_chebi,in_c3p,agreement_c3p_curator
3,CHEBI:47622,acetate ester,CCCCCCCCCCCCCC(=O)OC[C@H](COP([O-])(=O)OC[C@@H...,CCCCCCCCCCCCCC(=O)OC[C@H](COP([O-])(=O)OC[C@@H...,1-myristoyl-2-oleoyl-sn-glycero-3-phosphatidyl...,"NO, the statement is not true. The structure...","NO, the provided statement is not true. Expla...",NO. The structure 1-myristoyl-2-oleoyl-sn-glyc...,NO Explanation: The provided SMILES string 'C...,"NO, the provided statement is incorrect. 1-m...",No,It is a carboxylic ester,False,False,True,True,True,False,True
8,CHEBI:17478,aldehyde,CCCCCCCCCCCCCC(=O)OC[C@H](COP([O-])(=O)OC[C@@H...,CCCCCCCCCCCCCC(=O)OC[C@H](COP([O-])(=O)OC[C@@H...,1-myristoyl-2-oleoyl-sn-glycero-3-phosphatidyl...,"NO, the statement is incorrect. 1-Myristoyl-2...",NO. 1-myristoyl-2-oleoyl-sn-glycero-3-phosphat...,NO. The structure 1-myristoyl-2-oleoyl-sn-glyc...,NO Explanation: The provided compound is a ty...,"NO, the statement is not true. The structure 1...",No,It is a carboylic ester,False,False,True,True,True,False,True
9,CHEBI:17478,aldehyde,C(=C\C/C=C\CCCC(NC)=O)\C/C=C\C/C=C\CCCCC,C(=C\C/C=C\CCCC(NC)=O)\C/C=C\C/C=C\CCCCC,N-methyl arachidonoyl amine,"NO, the provided statement is not true. The ...","NO. The provided structure, N-methyl arachidon...",NO. The structure N-methyl arachidonoyl amine ...,NO The provided structure N-methyl arachidono...,"NO, the provided statement is not true. The ...",No,It is a secondary carboxamide,False,False,True,True,True,False,True
10,CHEBI:17478,aldehyde,O=C1C2=C(OC(=C1)C)C3=C(OC)C=C(OC)C=C3C(=C2O)C4...,No change,Isonigerone,"NO, Isonigerone is not a type of aldehyde. The...",NO. Isonigerone is not an aldehyde. Looking at...,"NO. The structure known as Isonigerone, as rep...",NO The provided structure for Isonigerone is ...,"NO, the provided statement is incorrect. The S...",No,It is a polycyclic compound,False,False,True,True,True,False,True
12,CHEBI:2571,aliphatic alcohol,C[C@H]1CN(C(=O)C2=C(C3=CC=CC=C3CO[C@@H]1CN(C)C...,No change,LSM-30659,"NO, the provided statement is not true. Based ...",NO. The provided statement is incorrect. Expl...,"NO. The structure LSM-30659, as described by t...",NO Explanation: The provided structure LSM-30...,"NO, the statement is incorrect. The structure ...",No,the structure as a whole is not aliphatic,False,False,True,True,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,CHEBI:36141,quinone,CC[C@@]1(C)CC(=O)NC1=O,No change,(S)-ethosuximide,"NO, the statement is not true. (S)-Ethosuxim...",NO. The provided structure (S)-ethosuximide (w...,NO. The structure (S)-ethosuximide (SMILES 'CC...,NO Explanation: The provided structure for (S...,"NO, the provided statement is not true. (S)-E...",No,NaN,False,False,True,True,True,False,True
205,CHEBI:17855,triglyceride,CCOc1nc2cccc(C(=O)OC(C)OC(=O)OC3CCCCC3)c2n1Cc1...,No change,Candesartan cilexetil,"NO, Candesartan cilexetil is not a type of tri...","NO, the provided statement is not true. Cande...",NO. The structure known as Candesartan cilexet...,NO Candesartan cilexetil is not a type of tri...,"NO, the provided statement is false. The chemi...",No,NaN,False,False,True,True,True,False,True
206,CHEBI:17855,triglyceride,CCCCCCCCCCCCCCCC(=O)OCC(CSC[C@H](N)C(O)=O)OC(=...,No change,"2,3-dipalmitoyl-S-glycerylcysteine","NO, the structure described as 2,3-dipalmitoyl...",NO. The provided SMILES string 'CCCCCCCCCCCCCC...,"NO. The structure 2,3-dipalmitoyl-S-glycerylcy...",NO Explanation: The structure provided is not...,"NO, the provided statement is not true. The st...",No,NaN,False,False,True,True,True,False,True
214,CHEBI:27208,unsaturated fatty ac

In [77]:
curator_df[curator_df["agreement_benchmark_chebi"] == False] 

,chemical_class_id,chemical_class_name,smiles,fixed_smiles,structure_name,model=citeseek-gpt-4o_response,model=claude-sonnet_response,model=deepseek-chat_response,model=gpt-3.5-turbo_response,model=gpt-4o_response,Curator: is_member?,Curator: comments,in_benchmark,in_chebi,agreement_benchmark_chebi,agreement_curator_benchmark,agreement_curator_chebi,in_c3p,agreement_c3p_curator


In [78]:
curator_df.to_csv(f"{RESULTS_DIR}/manual-evals-with-c3p.csv", index=False)

In [79]:
curator_df.to_excel(f"{RESULTS_DIR}/manual-evals-with-c3p.xlsx", index=False)